In [20]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [21]:
# Load Person Validation Set

!wget https://raw.githubusercontent.com/Mandaranian-Cactus/MNIST-CNN/main/Personal%20Validation%20Set/2.png -P PersonalValidationSet
!wget https://raw.githubusercontent.com/Mandaranian-Cactus/MNIST-CNN/main/Personal%20Validation%20Set/1.png -P PersonalValidationSet
!wget https://raw.githubusercontent.com/Mandaranian-Cactus/MNIST-CNN/main/Personal%20Validation%20Set/3.png -P PersonalValidationSet
!wget https://raw.githubusercontent.com/Mandaranian-Cactus/MNIST-CNN/main/Personal%20Validation%20Set/4.png -P PersonalValidationSet
!wget https://raw.githubusercontent.com/Mandaranian-Cactus/MNIST-CNN/main/Personal%20Validation%20Set/6.png -P PersonalValidationSet
!wget https://raw.githubusercontent.com/Mandaranian-Cactus/MNIST-CNN/main/Personal%20Validation%20Set/5.png -P PersonalValidationSet
!wget https://raw.githubusercontent.com/Mandaranian-Cactus/MNIST-CNN/main/Personal%20Validation%20Set/7.png -P PersonalValidationSet
!wget https://raw.githubusercontent.com/Mandaranian-Cactus/MNIST-CNN/main/Personal%20Validation%20Set/8.png -P PersonalValidationSet
!wget https://raw.githubusercontent.com/Mandaranian-Cactus/MNIST-CNN/main/Personal%20Validation%20Set/9.png -P PersonalValidationSet


--2021-10-08 00:35:42--  https://raw.githubusercontent.com/Mandaranian-Cactus/MNIST-CNN/main/Personal%20Validation%20Set/2.png
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30673 (30K) [image/png]
Saving to: ‘PersonalValidationSet/2.png.3’

2.png.3             100%[===================>]  29.95K  --.-KB/s    in 0.003s  

2021-10-08 00:35:42 (8.90 MB/s) - ‘PersonalValidationSet/2.png.3’ saved [30673/30673]

--2021-10-08 00:35:42--  https://raw.githubusercontent.com/Mandaranian-Cactus/MNIST-CNN/main/Personal%20Validation%20Set/1.png
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP 

In [22]:
# import standard PyTorch modules
import torch
import torch.nn as nn  # Provides many classes and functions to build neural networks (Layer types, activation functions, etc.)
import torch.nn.functional as F  
import torch.optim as optim  # Provides pre-coded optimization methods (Momentum, SGD, ADAM, etc.)
from torch.utils.tensorboard import SummaryWriter # TensorBoard support (Allows the generation of visualization reports for training data, comparing results, etc.)

# import torchvision module to handle image manipulation. Also includes the MNIST dataset
import torchvision
import torchvision.transforms as transforms

# calculate train time, writing train data to files etc.
import time
import pandas as pd
import json
from IPython.display import clear_output

torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)     # On by default, leave it here for clarity


In [23]:
# Use standard MNIST dataset
trainSet = torchvision.datasets.MNIST(
    root = './data/MNIST/train',
    train = True,
    download = True,
    transform = transforms.Compose([
        transforms.ToTensor()                                 
    ])
)

testSet = torchvision.datasets.MNIST(
    root = './data/MNIST/test',
    train = False,
    download = True,
    transform = transforms.Compose([
        transforms.ToTensor()                                 
    ])
)

In [24]:
from PIL import Image
from PIL import ImageOps
import os

# Functions loads and transforms PIL image into a pytorch tensor (1 x w x h)
def imageLoader(image_name, w, h):
    loader = transforms.Compose([
      transforms.ToTensor()])  
    image = Image.open(image_name).convert('L')
    image = ImageOps.invert(image)
    image = image.resize((w, h))
    image = loader(image)
    image = image.unsqueeze(0)
    return image

# Extract images from "Personal MNIST Validation Set" folder and covnert to pytorch tensors
personalValidationSet = None
fileNames = []
for fileName in os.listdir("PersonalValidationSet"):
  if fileName.endswith(".png") or fileName.endswith(".PNG"): 
    fileNames.append(fileName)
    img = imageLoader(f"PersonalValidationSet/{fileName}", 28, 28)
    if personalValidationSet == None: personalValidationSet = img # Edge-case: Insert the 1st tenesor into the validation set manually
    else: personalValidationSet = torch.cat((personalValidationSet, img), 0) # Concactonate the tensors (Exp: combines two 1 x 28 x 28 tensors into one 2 x 28 x 28 tensor)


In [25]:
class Network(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=5)
    self.batchNorm1 = nn.BatchNorm2d(10)
    self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5)
    self.batchNorm2 = nn.BatchNorm2d(20)
    # self.fullyConnected1 = nn.Linear(in_features=20*4*4, out_features=120)
    # self.batchNorm3 = nn.BatchNorm1d(120)
    # self.dropOut1 = nn.Dropout(p=0.25)
    self.fullyConnected2 = nn.Linear(in_features=20*4*4, out_features=60)
    self.batchNorm4 = nn.BatchNorm1d(60)
    self.dropOut2 = nn.Dropout(p=0.25)
    self.output = nn.Linear(in_features=60, out_features=10)

  def forward(self, t):
    # Convolutional Layer 1
    t = self.conv1(t)
    t = self.batchNorm1(t)
    t = F.relu(t)
    t = F.max_pool2d(t, kernel_size=2, stride=2)

    # Convolutional layer 2
    t = self.conv2(t)
    t = self.batchNorm2(t)
    t = F.relu(t)
    t = F.max_pool2d(t, kernel_size=2, stride=2)

    # # Fully connected layer 1
    # t = t.reshape(-1, 20*4*4)
    # t = self.fullyConnected1(t)
    # t = self.batchNorm3(t)
    # t = F.relu(t)
    # t = self.dropOut1(t)

    # Fully connected layer 2
    t = t.reshape(-1, 20*4*4)
    t = self.fullyConnected2(t)
    t = self.batchNorm4(t)
    t = F.relu(t)
    t = self.dropOut2(t)

    # Fully connected layer 3
    t = self.output(t)

    # Loss function will be calulated using softmax cross-entropy. Will be done outside of the forward pass method

    return t

In [26]:
from collections import OrderedDict
from collections import namedtuple
from itertools import product

class RunBuilder():
  @staticmethod
  def getRuns(params):
    Run = namedtuple("run", params.keys())
    runs = []
    for run in product(*params.values()):
      runs.append(Run(*run))
    return runs

In [27]:
# Class which managed 
# 1. Store the runtimes for each run and its repsective epochs
# 2. Store the training data (accuracy, loss, gradients, weights, etc. ) for each run and epoch 
# 3. Store the data from 1. and 2. into a TensorBoard for further analysis
# 4. Store the data from 1. and 2. into a csv and json file for further reference of API extraction
class RunManager():
  def __init__(self):
    # Tracks each epoch's count, loss, accuracy, and duration
    self.epochCnt = 0
    self.epochLoss = 0
    self.epochNumCorrect = 0
    self.epochStartTime = 0
  
    # Tracks each run's count, hyperparameters (Learning rate, batchSize, etc.), data, duration, and validation set accuracy
    self.runCnt = 0
    self.runParams = None
    self.runData = []  # Contains all iterations of the "epochData"
    self.runStartTime = 0
    self.runValidationNumCorrect = 0

    # Records neural network model, MNIST dataset, and TensorBoard for storing data
    self.network = None
    self.trainLoader = None
    self.testLoader = None
    self.tb = None

  def setTestLoader(self, testLoader):
    self.testLoader = testLoader

  def beginRun(self, runParams, network, trainLoader):
    self.runStartTime = time.time()
    self.runParams = runParams
    self.runCnt += 1

    self.network = network
    self.trainLoader = trainLoader
    self.tb = SummaryWriter(comment=f'-{run}')

    images, labels = next(iter(self.trainLoader)) # Subsample a mini-batch of the "trainLoader" to be placed as a demo visualization in tensor board
    grid = torchvision.utils.make_grid(images) # Mashes the images together side by side to create a grid of images

    self.tb.add_image('images', grid)
    self.tb.add_graph(self.network, images)

  def endRun(self):

    runDuration = time.time() - self.runStartTime
    accuracy = self.runValidationNumCorrect / len(self.testLoader.dataset)
    results = {
              "run":self.runCnt,
              "epoch":"NULL",
              "loss":"NULL",
              "accuracy":accuracy,
              "epochDuration":"NULL",
              "runDuration":runDuration}

    # Upload hyper-parameters into "results"    
    for k, v in self.runParams._asdict().items(): results[k] = v
    self.runData.append(results)
    df = pd.DataFrame.from_dict(self.runData, orient='columns')

    # Display epoch information and show progress
    clear_output() # Clear the previous dataframe from the console output
    display(df) # Display new dataframe in the console output

    self.tb.close()
    self.epochCnt = 0
    self.run_test_num_correct = 0
    self.runValidationAccuracy = 0

  def beginEpoch(self):
    self.epochStartTime = time.time()
    self.epochCnt += 1
    self.epochLoss = 0
    self.epochNumCorrect = 0

  def endEpoch(self):
    # Calculate epoch runtime and run runtime (Accumulate)
    epochDuration = time.time() - self.epochStartTime
    runDuration = time.time() - self.runStartTime

    # Cross entropy loss (https://datascience.stackexchange.com/questions/20296/cross-entropy-loss-explanation)
    loss = self.epochLoss / len(self.trainLoader.dataset) # Computes cross entropy loss (epochLoss is the total loss across the entire training set over 1 epoch. We average out the loss here by diving by # of images in training set)
    accuracy = self.epochNumCorrect / len(self.trainLoader.dataset)

    # Record epoch loss and accuracy to TensorBoard 
    self.tb.add_scalar('Loss', loss, self.epochCnt)
    self.tb.add_scalar('Accuracy', accuracy, self.epochCnt)

    # Record params to TensorBoard
    for name, param in self.network.named_parameters():
      self.tb.add_histogram(name, param, self.epochCnt)
      self.tb.add_histogram(f'{name}.grad', param.grad, self.epochCnt)

    # Write all epoch related data into "results" OrderedDict
    results = {
        "run":self.runCnt,
        "epoch":self.epochCnt,
        "loss":loss,
        "accuracy":accuracy,
        "epochDuration":epochDuration,
        "runDuration":runDuration,
    }
    # Record hyper-parameters into "results"
    for k,v in self.runParams._asdict().items(): results[k] = v
    self.runData.append(results)
    df = pd.DataFrame.from_dict(self.runData, orient = 'columns')

    # Display epoch information and show progress
    clear_output()
    display(df)

  def calculateNumCorrect(self, preds, labels):
    numCorrect = preds.argmax(axis=1).eq(labels).sum().item()
    return numCorrect

  def trackEpochNumCorrect(self, preds, labels):
    # Given 2D prediction matrix (Each row contains 1 input cases' predictions) and labels vector, compute the # of correct predictions
    numCorrect = self.calculateNumCorrect(preds, labels)
    self.epochNumCorrect += numCorrect

  def trackRunNumValidationCorrect(self, preds, labels):
    numCorrect = self.calculateNumCorrect(preds, labels)
    self.runValidationNumCorrect = numCorrect

  # Accumulate loss for entire epoch
  # Recieves the average loss of a batch. Mutliply average batch loss by batch size in order to get total batch loss 
  def trackLoss(self, loss):
    self.epochLoss += loss.item() * self.trainLoader.batch_size

  # save end results of all runs into csv, json for further analysis
  def save(self, fileName):

    pd.DataFrame.from_dict(
        self.runData, 
        orient = 'columns',
    ).to_csv(f'{fileName}.csv')

    with open(f'{fileName}.json', 'w', encoding='utf-8') as f:
      json.dump(self.runData, f, ensure_ascii=False, indent=4)
  


In [ ]:
m = RunManager()
networks = []
params = OrderedDict( 
                   lr=[0.001, 0.01],
                   batchSize=[100, 1000],
                   shuffle=[True, False])
# params = OrderedDict( 
#                    lr=[0.01],
#                    batchSize=[30],
#                    shuffle=[True])
epochs = 3


testLoader = torch.utils.data.DataLoader(testSet, batch_size=10000)
m.setTestLoader(testLoader)
for run in RunBuilder.getRuns(params):
  network = Network()
  trainLoader = torch.utils.data.DataLoader(trainSet, batch_size=run.batchSize, shuffle=run.shuffle)
  print(len(trainLoader), trainLoader.batch_size)
  optimizer = optim.Adam(network.parameters(), lr=run.lr)

  m.beginRun(run, network, trainLoader)
  for epoch in range(epochs):
    m.beginEpoch()
    for batch in trainLoader:
      network.eval() # Set model to evaluation mode
      images = batch[0]
      labels = batch[1]
      preds = network(images)
      loss = F.cross_entropy(preds, labels)

      network.train() # Set model to training mode
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      m.trackEpochNumCorrect(preds, labels)
      m.trackLoss(loss)

    m.endEpoch()

  for batch in testLoader:
    network.eval() # Set model to evaluation mode
    images = batch[0]
    labels = batch[1]
    preds = network(images)
    m.trackRunNumValidationCorrect(preds, labels)

  networks.append(network)
  m.endRun()
m.save("Saved")



,run,epoch,loss,accuracy,epochDuration,runDuration,lr,batchSize,shuffle
0,1,1,0.339230,0.900667,28.720166,28.938510,0.001,100,True
1,1,2,0.091329,0.972083,29.750858,58.832733,0.001,100,True


In [ ]:
torch.save(networks[-1].state_dict(), "Model") # Save trained neural network

In [ ]:
# Evaluate personal validation set

# Load CNN
completedModel = Network()
completedModel.load_state_dict(torch.load("Model"))
completedModel.eval() # Switch from training to evaluation mode

# Personal validation set predictions

pred = completedModel(personalValidationSet)
predClass = pred.argmax(axis=1)

# Display predictions
print("Ground Truth: ")
print(fileNames)
print("Predictions: ")
print(predClass)
print("All Scores")
print(pred)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs
